In [42]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

warnings.filterwarnings("ignore")

# 4.3.1	Функция «Динамики бронирования вылетевшего рейса»

In [2]:
dates = ['FLTDAT', 'DAIS']
dtypes = {'FLTNUM': 'int16', 'TARIF_RUB': 'float16'}
bb = pd.read_csv('F:\\SVO-AER_2018-01-01--2019-12-31BB.csv',
                 delimiter=';', dtype=dtypes, parse_dates=dates)
bb = bb.dropna(how='any', axis=0)
bb.drop_duplicates(inplace=True)
bb['DEPARTURE'] = bb['FLTDAT'] + pd.to_timedelta(bb['FLTTIME'])
bb = bb[['FLTDAT', 'DEPARTURE', 'FLTNUM']].drop_duplicates()

use_cols = ('SDAT_S', 'FLT_NUM', 'DD', 'SSCL1', 'SEG_CLASS_CODE', 'FCLCLD',
            'PASS_BK', 'PASS_DEP', 'NS', 'DTD')
dates = ['SDAT_S', 'DD']
dtypes = {'FLT_NUM': 'int16', 'FCLCLD': 'int16', 'PASS_BK': 'int16','SA': 'int16',
          'AU': 'int16', 'PASS_DEP': 'int16', 'NS': 'int16', 'DTD': 'int16'}
hh = pd.read_csv('F:\\hh_for_profiles.csv', dtype=dtypes, parse_dates=dates, usecols=use_cols)

In [77]:
def get_flight(fltnum: int, date: datetime, flttime: pd.Timedelta) -> (str, pd.DataFrame):
    """Получение данных рейса для восстановления спроса"""
    departure = date + flttime
    flight = bb[(bb['FLTNUM'] == fltnum) & (bb['DEPARTURE'] == departure)]
    print(f"Рейс:", fltnum, departure)
    flight = pd.merge(flight, hh, left_on=['FLTNUM', 'FLTDAT'], right_on=['FLT_NUM', 'DD'])
    flight = flight.dropna(how='any', axis=0)
    flight.drop(columns=['FLTDAT', 'FLT_NUM', 'DD'], inplace=True)
    flight.drop_duplicates(inplace=True)
    print(f"Данных: {flight.shape[0]:_}")
    return f"{fltnum}, {departure}", flight

def flight_graph(flight: pd.DataFrame, flight_name: str) -> None:
    """График бронирования билетов для всего рейса"""
    flight = pd.DataFrame(flight.groupby(by='DTD')['PASS_BK'].sum()).reset_index()
    flight['DTD'] = -flight['DTD']
    fig = px.line(flight, x="DTD", y="PASS_BK", title=f"График спроса для рейса {flight_name}")
    fig.update_layout(xaxis_title="День до вылета", yaxis_title="Количество забронированных билетов")
    fig.update_xaxes(tickvals=list(range(flight['DTD'].min(), 2, 14)))
    fig.show()

def cabin_graph(flight: pd.DataFrame, flight_name: str) -> None:
    """График продаж по кабинам"""
    flight = pd.DataFrame(flight.groupby(by=['SSCL1', 'DTD'])['PASS_BK'].sum()).reset_index()
    flight['DTD'] = -flight['DTD']
    fig = px.line(flight, x="DTD", y="PASS_BK", color='SSCL1',
                  title=f"График спроса для рейса {flight_name}")
    fig.update_layout(xaxis_title="День до вылета", yaxis_title="Количество забронированных билетов")
    fig.update_xaxes(tickvals=list(range(flight['DTD'].min(), 2, 14)))
    fig.show()

def class_graph(flight: pd.DataFrame, flight_name: str) -> None:
    """График продаж по классам"""
    flight = pd.DataFrame(flight.groupby(by=['SSCL1', 'SEG_CLASS_CODE', 'DTD'])['PASS_BK'].sum())
    flight = flight.reset_index()
    flight['DTD'] = -flight['DTD']
    cabin_C = flight[flight['SSCL1'] == 'C']
    cabin_Y = flight[flight['SSCL1'] == 'Y']
    fig = px.line(cabin_C, x="DTD", y="PASS_BK", color='SEG_CLASS_CODE',
                  title=f"График спроса для рейса {flight_name} кабины C")
    fig.update_layout(xaxis_title="День до вылета", yaxis_title="Количество забронированных билетов")
    fig.update_xaxes(tickvals=list(range(flight['DTD'].min(), 2, 14)))
    fig.show()
    fig = px.line(cabin_Y, x="DTD", y="PASS_BK", color='SEG_CLASS_CODE',
                  title=f"График спроса для рейса {flight_name} кабины Y")
    fig.update_layout(xaxis_title="День до вылета", yaxis_title="Количество забронированных билетов")
    fig.update_xaxes(tickvals=list(range(flight['DTD'].min(), 2, 14)))
    fig.show()

In [4]:
%%time
flight_name, flight = get_flight(1122,
                                 datetime.strptime('2018-01-06', '%Y-%m-%d'),
                                 pd.Timedelta('09:10:00'))

Рейс: 1122 2018-01-06 09:10:00
Данных: 4_158
Wall time: 2.25 s


In [80]:
flight_graph(flight, flight_name)

In [78]:
cabin_graph(flight, flight_name)

In [79]:
class_graph(flight, flight_name)